In [2]:
pip install fastapi uvicorn joblib pandas scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from fastapi import FastAPI, HTTPException
import uvicorn
import joblib
import pandas as pd
import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

# ✅ Aplicar nest_asyncio si ejecutas en Jupyter Notebook
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    pass

# ✅ Ruta completa del modelo
modelo_path = r"C:\Users\AsusTUF\Documents\taller\modelo\api\mejor_modelo.gz"  # Ajusta si está en otro lugar

# ✅ Cargar el modelo al iniciar la API
if os.path.exists(modelo_path):
    try:
        model = joblib.load(modelo_path)
        print(f"✅ Modelo cargado exitosamente desde: {modelo_path}")
    except Exception as e:
        print(f"❌ Error al cargar el modelo: {e}")
        model = None
else:
    print(f"❌ El archivo '{modelo_path}' no existe. Verifica la ruta.")
    model = None

# ✅ Definir las variables seleccionadas para el clustering
score_columns = [
    'TOTAL_OPERATING_REVENUE', 'CASH_AND_CASH_EQUIVALENTS', 'EQUITY_ATTRIBUTABLE_TO_OWNERS_OF_THE_PARENT',
    'ISSUED_CAPITAL', 'CURRENT_LIABILITIES_TO_TOTAL_LIABILITIES_PERCENTAGE', 'RETURN_ON_CAPITAL_EMPLOYED_PERCENTAGE',
    'WORKING_CAPITAL', 'NET_PROFIT_MARGIN_PERCENTAGE', 'PROPERTY_PLANT_AND_EQUIPMENT', 'ADMINISTRATIVE_EXPENSES_NET_SALES_PERCENTAGE',
    'NON_CURRENT_ASSET_TURNOVER_X', 'TRADE_RECEIVABLE_TURNOVER_X', 'NET_CASH', 'DEBT_RATIO', 'TOTAL_ASSETS_TREND_PERCENTAGE',
    'SHAREHOLDERS_EQUITY_TREND_PERCENTAGE', 'ALTMAN_Z_SCORE', 'GROSS_PROFIT_TREND_PERCENTAGE', 'TOTAL_OPERATING_REVENUE_TREND_PERCENTAGE',
    'NET_CASH_FLOW_FROM_USED_IN_OPERATING_ACTIVITIES', 'RETAINED_EARNINGS', 'CASH_RATIO_X', 'EBITDA', 'INCOME_TAX',
    'OPERATING_PROFIT_TREND_PERCENTAGE', 'NET_PROFIT_TREND_PERCENTAGE', 'DEBT_TO_EBITDA_X', 'NET_CASH_FLOW_FROM_USED_IN_INVESTING_ACTIVITIES',
    'OPERATING_CASH_FLOW_TO_DEBT_X', 'SHARE_PREMIUM', 'NET_PROPERTY_PLANT_AND_EQUIPMENT_PPYE_TREND_PERCENTAGE', 'SHORT_TERM_DEBT',
    'INTANGIBLE_ASSETS_AND_GOODWILL', 'EBITDA_TREND_PERCENTAGE', 'INVENTORY_TREND_PERCENTAGE'
]

# ✅ Inicializar el **escalador MinMaxScaler** y PCA con los mismos parámetros que en el entrenamiento
scaler = MinMaxScaler()
pca = PCA(n_components=1)  # Asegúrate de usar el mismo número de componentes que en el entrenamiento

# ✅ Iniciar FastAPI
app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "API de Clustering con FastAPI está funcionando correctamente 🚀"}

@app.post("/predict/")
def predict_cluster(data: dict):
    """
    Endpoint para predecir el clúster de un nuevo registro basado en K-Means con PCA.
    """
    if model is None:
        raise HTTPException(status_code=500, detail=f"El modelo no está disponible en '{modelo_path}'.")

    try:
        # ✅ Convertir los datos a DataFrame
        df_input = pd.DataFrame([data])

        # ✅ Asegurar que todas las columnas requeridas están presentes
        for col in score_columns:
            if col not in df_input.columns:
                df_input[col] = 0  # Rellenar valores faltantes con 0

        # ✅ Ordenar las columnas en el mismo orden que en el entrenamiento
        df_input = df_input[score_columns]

        # ✅ Aplicar **MinMaxScaler** (usando los mismos valores que en el entrenamiento)
        df_scaled = scaler.fit_transform(df_input)  # Usa `fit_transform` por el el escalador

        # ✅ Aplicar **PCA** (debes haber usado el mismo número de componentes)
        df_pca = pca.fit_transform(df_scaled)

        # ✅ Hacer la predicción con el modelo K-Means
        cluster = model.predict(df_pca)
        return {"cluster_predicho": int(cluster[0])}

    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error en la predicción: {str(e)}")


# ✅ Para ejecutar la API
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


✅ Modelo cargado exitosamente desde: C:\Users\AsusTUF\Documents\taller\modelo\api\mejor_modelo.gz


INFO:     Started server process [57184]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:54791 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54791 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:54929 - "POST /predict/ HTTP/1.1" 200 OK


c:\Users\AsusTUF\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:591: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
